# Build Docker image 

In this notebook, we will build the docker container that contains the ResNet152 model, Flask web application, model driver and all dependencies.
Make sure you have logged in using docker login.

In [1]:
import os
from os import path
import json
import shutil
%load_ext dotenv

We will be using the following Docker information to push the image to docker hub.

In [2]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name
docker_login = "YOUR_DOCKER_LOGIN"
image_repo = "/kerastf-gpu"

Overwriting .env


In [4]:
os.makedirs('flaskwebapp', exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'nginx'), exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'etc'), exist_ok=True)

In [5]:
shutil.copy('resnet152.py', 'flaskwebapp')
shutil.copy('driver.py', 'flaskwebapp')
os.listdir('flaskwebapp')

['resnet152.py', 'etc', 'driver.py', 'nginx']

Below, we create the module for the Flask web application.

In [6]:
%%writefile flaskwebapp/app.py

from flask import Flask, request
import logging
import json
import driver

app = Flask(__name__)
predict_for = driver.get_model_api()
 
@app.route("/score", methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    request_input = request.json['input']
    response = predict_for(request_input)
    print(response)
    return json.dumps({'result': str(response)})


@app.route("/")
def healthy():
    return "Healthy"

# Tensorflow Version
@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

Writing flaskwebapp/app.py


In [7]:
%%writefile flaskwebapp/wsgi.py
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


Here, we write the configuration for the Nginx which creates a proxy between ports **80** and **5000**.

In [8]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


In [9]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Writing flaskwebapp/gunicorn_logging.conf


In [10]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()

Writing flaskwebapp/kill_supervisor.py


In [11]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We create a custom image based on the CUDA 9 image from NVIDIA and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [12]:
%%writefile flaskwebapp/requirements.txt

Pillow==5.0.0
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.18.4
h5py==2.6.0

Writing flaskwebapp/requirements.txt


In [13]:
%%writefile flaskwebapp/dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
    
RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

USER root
RUN mkdir /code
WORKDIR /code
RUN chmod -R a+w /code
ADD . /code/

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        wget \
        curl \
        git \
        supervisor \
        nginx && \
    rm -rf /var/lib/apt/lists/* &&\
    rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ 

ENV PYTHON_VERSION=3.5
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy pyyaml scipy \
    ipython pandas jupyter ipykernel scikit-learn && \
    /opt/conda/bin/conda clean -ya
    
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
    
RUN pip install --upgrade pip && \
    pip install tensorflow-gpu==1.9.0 && \
    pip install keras==2.2.0 && \
    pip install -r /code/requirements.txt && \       
    /opt/conda/bin/conda clean -yt

EXPOSE 8888
EXPOSE 5000
EXPOSE 80

CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

Writing flaskwebapp/dockerfile


The image name below refers to our dockerhub account. If you wish to push the image to your account make sure you change the docker login.

In [14]:
%dotenv
image_name = os.getenv('docker_login') + os.getenv('image_repo')
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

Next, we build our docker image. The output of this cell is cleared from this notebook as it is quite long due to all the installations required to build the image. However, you should make sure you see 'Successfully built' and 'Successfully tagged' messages in the last line of the output when you run the cell. 

In [ ]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Below we will push the image created to our dockerhub registry. Make sure you have already logged in to the appropriate dockerhub account using the docker login command. If you haven't loged in to the approrpiate dockerhub account you will get an error.

In [17]:
!docker push $image_name

The push refers to repository [docker.io/caia/kerastf-gpu]

52711ebe: Preparing 
3f9bf5d6: Preparing 
de757249: Preparing 
300a73bf: Preparing 
8c66fc09: Preparing 
b97e3194: Preparing 
0b9c5c51: Preparing 
a273db2b: Preparing 
35faacf3: Preparing 
6f924f59: Preparing 
5cb8493b: Preparing 
62ee5fac: Preparing 
28798916: Preparing 
dd00b1a4: Preparing 
c3c04cbd: Preparing 
daf493f1: Preparing 
88d0e278: Preparing 


f9bf5d6: Pushing  999.4MB/2.059GBet-gpu 1KPushing  330.4MB/2.059GBPushing  654.3MB/2.059GBPushing  1.027GB/1.263GBf9bf5d6: Pushed   2.137GB/2.059GBPushing  1.963GB/2.059GBlatest: digest: sha256:029335f63f0f52e21f6ad2e5efbf966e1b47c36eaf90812b5fc646667d1779bb size: 4090


In [18]:
print('Docker image name {}'.format(image_name))

Docker image name caia/kerastf-gpu


We can now [test our image locally](03_TestLocally.ipynb).